In [646]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np
import docplex

In [647]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [(("-"if _i == -1 else "" if _i == 1 else (str(_i) + "*")) + n) for _i, n in zip(c[0], names) if _i != 0]
        print(c[3] + ": ([" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2]) + ")") 
        

In [648]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = ["1","2","3","4","5"]                                # V
customers = ["1","2","3","4","5","6","7","8","9"]                       # C

num_vehicles = 5
num_customers = 7
num_locations = num_customers + 1                 
num_names = (num_locations ** 2) * num_vehicles

locations = ["O"] + customers[:num_customers]         # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route

In [649]:
my_names = ["x"+str(i)+str(j) + str(k) for k in vehicles[:num_vehicles] for i in locations[:num_locations] for j in locations[:num_locations]]  

print(len(my_names))
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

320
Names  ['xOO1', 'xO11', 'xO21', 'xO31', 'xO41', 'xO51', 'xO61', 'xO71', 'x1O1', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x171', 'x2O1', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x271', 'x3O1', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x371', 'x4O1', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x471', 'x5O1', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x571', 'x6O1', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x671', 'x7O1', 'x711', 'x721', 'x731', 'x741', 'x751', 'x761', 'x771', 'xOO2', 'xO12', 'xO22', 'xO32', 'xO42', 'xO52', 'xO62', 'xO72', 'x1O2', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x172', 'x2O2', 'x212', 'x222', 'x232', 'x242', 'x252', 'x262', 'x272', 'x3O2', 'x312', 'x322', 'x332', 'x342', 'x352', 'x362', 'x372', 'x4O2', 'x412', 'x422', 'x432', 'x442', 'x452', 'x462', 'x472', 'x5O2', 'x512', 'x522', 'x532', 'x542', 'x552', 'x562', 'x572', 'x6O2', 'x612', 'x622', 'x632', 'x642', 'x652', 'x662', 'x672', 'x7O2', 'x712', 'x722', 'x73

In [650]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
__v = lambda n: 0 if n == "O" else int(n)
my_obj = [int(cost_matrix.iat[__v(n[1]),__v(n[2]) + 1]) for n in my_names]

print("Object ",my_obj)

Object  [0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 18, 22, 13, 30, 23, 0, 32, 29, 18, 10, 26, 28, 10, 32, 0, 2, 20, 31, 17, 27, 4, 29, 2, 0, 19, 28, 12, 8, 18, 18, 20, 19, 0, 5, 8, 18, 22, 10, 31, 28, 5, 0, 17, 5, 13, 26, 17, 12, 8, 17, 0, 0, 18, 30, 28, 27, 8, 18, 5, 18, 0, 23, 10, 4, 

In [651]:
capacity_matrix = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[100, 70, 145, 90, 50]

In [652]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{'1': 32, '2': 45, '3': 16, '4': 15, '5': 22, '6': 50, '7': 33}

In [653]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [654]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
constraint_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if name[1] != name[2]  and name[1] == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        constraint_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(constraint_2)


C21: ([x1O1 x121 x131 x141 x151 x161 x171 x1O2 x122 x132 x142 x152 x162 x172 x1O3 x123 x133 x143 x153 x163 x173 x1O4 x124 x134 x144 x154 x164 x174 x1O5 x125 x135 x145 x155 x165 x175]  E  1)
C22: ([x2O1 x211 x231 x241 x251 x261 x271 x2O2 x212 x232 x242 x252 x262 x272 x2O3 x213 x233 x243 x253 x263 x273 x2O4 x214 x234 x244 x254 x264 x274 x2O5 x215 x235 x245 x255 x265 x275]  E  1)
C23: ([x3O1 x311 x321 x341 x351 x361 x371 x3O2 x312 x322 x342 x352 x362 x372 x3O3 x313 x323 x343 x353 x363 x373 x3O4 x314 x324 x344 x354 x364 x374 x3O5 x315 x325 x345 x355 x365 x375]  E  1)
C24: ([x4O1 x411 x421 x431 x451 x461 x471 x4O2 x412 x422 x432 x452 x462 x472 x4O3 x413 x423 x433 x453 x463 x473 x4O4 x414 x424 x434 x454 x464 x474 x4O5 x415 x425 x435 x455 x465 x475]  E  1)
C25: ([x5O1 x511 x521 x531 x541 x561 x571 x5O2 x512 x522 x532 x542 x562 x572 x5O3 x513 x523 x533 x543 x563 x573 x5O4 x514 x524 x534 x544 x564 x574 x5O5 x515 x525 x535 x545 x565 x575]  E  1)
C26: ([x6O1 x611 x621 x631 x641 x651 x671 x6O2 x61

In [655]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
constraint_3 = []
val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if n1 != "O" and n1 != n2 else c if n1 == "O" and n2 != "O" else 0
# val = lambda v, c, n1, n2, n3: 0 if n3 != v else int(demand[n1]) if n1 != "O" and n1 != n2 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    constraint_3.append(([val(veh, float(cap), name[1], name[2], name[3]) for name in my_names],"G", 0, "C3" + str(_i)))


display(constraint_3)

C31: ([100.0*xO11 100.0*xO21 100.0*xO31 100.0*xO41 100.0*xO51 100.0*xO61 100.0*xO71 -32*x1O1 -32*x121 -32*x131 -32*x141 -32*x151 -32*x161 -32*x171 -45*x2O1 -45*x211 -45*x231 -45*x241 -45*x251 -45*x261 -45*x271 -16*x3O1 -16*x311 -16*x321 -16*x341 -16*x351 -16*x361 -16*x371 -15*x4O1 -15*x411 -15*x421 -15*x431 -15*x451 -15*x461 -15*x471 -22*x5O1 -22*x511 -22*x521 -22*x531 -22*x541 -22*x561 -22*x571 -50*x6O1 -50*x611 -50*x621 -50*x631 -50*x641 -50*x651 -50*x671 -33*x7O1 -33*x711 -33*x721 -33*x731 -33*x741 -33*x751 -33*x761]  G  0)
C32: ([70.0*xO12 70.0*xO22 70.0*xO32 70.0*xO42 70.0*xO52 70.0*xO62 70.0*xO72 -32*x1O2 -32*x122 -32*x132 -32*x142 -32*x152 -32*x162 -32*x172 -45*x2O2 -45*x212 -45*x232 -45*x242 -45*x252 -45*x262 -45*x272 -16*x3O2 -16*x312 -16*x322 -16*x342 -16*x352 -16*x362 -16*x372 -15*x4O2 -15*x412 -15*x422 -15*x432 -15*x452 -15*x462 -15*x472 -22*x5O2 -22*x512 -22*x522 -22*x532 -22*x542 -22*x562 -22*x572 -50*x6O2 -50*x612 -50*x622 -50*x632 -50*x642 -50*x652 -50*x672 -33*x7O2 -33

In [656]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
constraint_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    constraint_4.append(([(1 if (name[1] == "O" and name[3] == k) else 0) for name in my_names],"L", 1, "C4" + str(_i)))
    
display(constraint_4)

C41: ([xOO1 xO11 xO21 xO31 xO41 xO51 xO61 xO71]  L  1)
C42: ([xOO2 xO12 xO22 xO32 xO42 xO52 xO62 xO72]  L  1)
C43: ([xOO3 xO13 xO23 xO33 xO43 xO53 xO63 xO73]  L  1)
C44: ([xOO4 xO14 xO24 xO34 xO44 xO54 xO64 xO74]  L  1)
C45: ([xOO5 xO15 xO25 xO35 xO45 xO55 xO65 xO75]  L  1)


In [657]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
constraint_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if n[3] != k or n[2] == n[1] else 1 if n[1] == p else -1 if n[2] == p else 0                
        constraint_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(constraint_5) 

C51: ([-xO11 x1O1 x121 x131 x141 x151 x161 x171 -x211 -x311 -x411 -x511 -x611 -x711]  E  0)
C52: ([-xO21 -x121 x2O1 x211 x231 x241 x251 x261 x271 -x321 -x421 -x521 -x621 -x721]  E  0)
C53: ([-xO31 -x131 -x231 x3O1 x311 x321 x341 x351 x361 x371 -x431 -x531 -x631 -x731]  E  0)
C54: ([-xO41 -x141 -x241 -x341 x4O1 x411 x421 x431 x451 x461 x471 -x541 -x641 -x741]  E  0)
C55: ([-xO51 -x151 -x251 -x351 -x451 x5O1 x511 x521 x531 x541 x561 x571 -x651 -x751]  E  0)
C56: ([-xO61 -x161 -x261 -x361 -x461 -x561 x6O1 x611 x621 x631 x641 x651 x671 -x761]  E  0)
C57: ([-xO71 -x171 -x271 -x371 -x471 -x571 -x671 x7O1 x711 x721 x731 x741 x751 x761]  E  0)
C58: ([-xO12 x1O2 x122 x132 x142 x152 x162 x172 -x212 -x312 -x412 -x512 -x612 -x712]  E  0)
C59: ([-xO22 -x122 x2O2 x212 x232 x242 x252 x262 x272 -x322 -x422 -x522 -x622 -x722]  E  0)
C510: ([-xO32 -x132 -x232 x3O2 x312 x322 x342 x352 x362 x372 -x432 -x532 -x632 -x732]  E  0)
C511: ([-xO42 -x142 -x242 -x342 x4O2 x412 x422 x432 x452 x462 x472 -x542 -x642 

In [658]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
constraint_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    constraint_6.append(([(1 if (name[3] == veh and name[2] == "O" and name[1] != "O") else 0) for name in my_names],"L", 1, "C6" + str(_i)))
    
display(constraint_6)


C61: ([x1O1 x2O1 x3O1 x4O1 x5O1 x6O1 x7O1]  L  1)
C62: ([x1O2 x2O2 x3O2 x4O2 x5O2 x6O2 x7O2]  L  1)
C63: ([x1O3 x2O3 x3O3 x4O3 x5O3 x6O3 x7O3]  L  1)
C64: ([x1O4 x2O4 x3O4 x4O4 x5O4 x6O4 x7O4]  L  1)
C65: ([x1O5 x2O5 x3O5 x4O5 x5O5 x6O5 x7O5]  L  1)


In [659]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
constraint_x = []
_i = 0

_i += 1
constraint_x.append(([(1 if name[1] == name[2] else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(constraint_x)


CX1: ([xOO1 x111 x221 x331 x441 x551 x661 x771 xOO2 x112 x222 x332 x442 x552 x662 x772 xOO3 x113 x223 x333 x443 x553 x663 x773 xOO4 x114 x224 x334 x444 x554 x664 x774 xOO5 x115 x225 x335 x445 x555 x665 x775]  E  0)


In [660]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
constraint_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            constraint_y.append(([(1 if ((name[1] == i and j == name[2]) or (name[1] == j and i == name[2])) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(constraint_y)


CX1: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX2: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX3: ([x141 x411 x142 x412 x143 x413 x144 x414 x145 x415]  L  1)
CX4: ([x151 x511 x152 x512 x153 x513 x154 x514 x155 x515]  L  1)
CX5: ([x161 x611 x162 x612 x163 x613 x164 x614 x165 x615]  L  1)
CX6: ([x171 x711 x172 x712 x173 x713 x174 x714 x175 x715]  L  1)
CX7: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX8: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX9: ([x241 x421 x242 x422 x243 x423 x244 x424 x245 x425]  L  1)
CX10: ([x251 x521 x252 x522 x253 x523 x254 x524 x255 x525]  L  1)
CX11: ([x261 x621 x262 x622 x263 x623 x264 x624 x265 x625]  L  1)
CX12: ([x271 x721 x272 x722 x273 x723 x274 x724 x275 x725]  L  1)
CX13: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX14: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX15: ([x341 x431 x342 x432 x343 x433 x344 x434 x345 x435]  L  1)
CX16: ([x351 x531 x

In [661]:
def foo(n : int, a : list, o : list):
    temp = {}
    tempa = []
    for ai in a:
        _i =  np.prod([ord(c) + 99 for c in ai])
        if _i not in temp:            
            temp[_i] = ai
            tempa.append(ai)
            
    a = tempa
    arr = []
    if n > 1:
        for ai in a:
            for oi in o:
                if oi not in ai:
                    arr.append(ai + oi)
        return foo(n-1, arr, o)
    else:
        return a

def get_path(n: int, a: list) -> list:
    return foo(n,a,a)

In [662]:
# # CONSTRAINT_Z 
constraint_z = []
_i = 0

for r in range(3,num_customers + 1):
    for p in get_path(r, customers[:num_customers]):
#         p = "134"
#         r = 3
        _i += 1
        constraint_z.append(([1 if name[1] in p and name[2] in p and name[1] != name[2] else 0 for name in my_names], "L", r-1, "CZ" + str(_i)))

display(constraint_z)


CZ1: ([x121 x131 x211 x231 x311 x321 x122 x132 x212 x232 x312 x322 x123 x133 x213 x233 x313 x323 x124 x134 x214 x234 x314 x324 x125 x135 x215 x235 x315 x325]  L  2)
CZ2: ([x121 x141 x211 x241 x411 x421 x122 x142 x212 x242 x412 x422 x123 x143 x213 x243 x413 x423 x124 x144 x214 x244 x414 x424 x125 x145 x215 x245 x415 x425]  L  2)
CZ3: ([x121 x151 x211 x251 x511 x521 x122 x152 x212 x252 x512 x522 x123 x153 x213 x253 x513 x523 x124 x154 x214 x254 x514 x524 x125 x155 x215 x255 x515 x525]  L  2)
CZ4: ([x121 x161 x211 x261 x611 x621 x122 x162 x212 x262 x612 x622 x123 x163 x213 x263 x613 x623 x124 x164 x214 x264 x614 x624 x125 x165 x215 x265 x615 x625]  L  2)
CZ5: ([x121 x171 x211 x271 x711 x721 x122 x172 x212 x272 x712 x722 x123 x173 x213 x273 x713 x723 x124 x174 x214 x274 x714 x724 x125 x175 x215 x275 x715 x725]  L  2)
CZ6: ([x131 x141 x311 x341 x411 x431 x132 x142 x312 x342 x412 x432 x133 x143 x313 x343 x413 x433 x134 x144 x314 x344 x414 x434 x135 x145 x315 x345 x415 x435]  L  2)
CZ7: ([x13

In [663]:
# Merge segment constraints
constraints_group = []
constraints_group += constraint_2 
constraints_group += constraint_3 
constraints_group += constraint_4 
constraints_group += constraint_5 
constraints_group += constraint_6
constraints_group += constraint_x
constraints_group += constraint_y
constraints_group += constraint_z


# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [664]:
# Create Cplex Problems
myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")


myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)
    
myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

range(0, 199)

In [665]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    for v, n in zip(values, names):
        if v != 0:
            print(n, " - ",v)
except cplex.exceptions.errors.CplexSolverError:
    print("No solution")

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 22 rows and 40 columns.
MIP Presolve modified 210 coefficients.
Reduced MIP has 177 rows, 280 columns, and 7770 nonzeros.
Reduced MIP has 280 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.64 ticks)
Found incumbent of value 228.000000 after 0.03 sec. (12.00 ticks)
Probing time = 0.00 sec. (3.44 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 177 rows, 280 columns, and 7770 nonzeros.
Reduced MIP has 280 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (3.66 ticks)
Probing time = 0.00 sec. (3.52 ticks)
Clique table members: 2318.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (3.95 ticks)

        Nodes                          